In [ ]:
#imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [ ]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY');

if not api_key:
    print("No API Key loaded")
elif not api_key.startswith("sk-proj-"):
    print("Your API Key does not start with sk-proj. Maybe it was copied wrongly. Please check it")
elif api_key.strip() != api_key:
    print("Your API Key does contain some spaces. Please remove them")
else: 
    print("Your API Key looks good")  

In [ ]:
openai = OpenAI(api_key=api_key);

In [ ]:
#Verify that the connection with OpenAI is working correctly
message = "Hello ChatGPT, I am calling you using your API. I made it!!!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content": message}])
response.choices[0].message.content

In [ ]:
class Website:
    """
    Utility class to scape webpage html content given the page url
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)    
        
        

In [ ]:
santa_clause_webside = Website("https://it.wikipedia.org/wiki/Santa_Clause")
print(santa_clause_webside.title)

In [ ]:
#System Prompt and User Prompt
system_prompt="You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
def user_prompt_for(website: Website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows;\n\
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
def messages_for(website: Website):
    return [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
#summarize fuction will get everything together
def summarize(url: str) -> str :
    website = Website(url);
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content 


In [ ]:
display(Markdown(summarize("https://it.wikipedia.org/wiki/Santa_Clause")))